In [2]:
import pandas as pd
import time
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import joblib
from datetime import date
import requests

/Users/matthew.yough/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
api_key = '5j8zrc2txw2xraj5uc6mw36g'

In [ ]:
# Read Old Dataset
cbb_raw_path = "/Users/matthew.yough/Documents/GitHub/ncaab_game_predictor"
cbb_raw_name = "cbb_raw_data.xlsx"

cbb_training_path = "/Users/matthew.yough/Documents/GitHub/ncaab_game_predictor"
cbb_training_name = "cbb_training_data.xlsx"

old_raw_df = pd.read_excel(f"{cbb_raw_path}/{cbb_raw_name}", index_col=0)
old_training_df = pd.read_excel(f"{cbb_training_path}/{cbb_training_name}", index_col=0)

In [ ]:
today = date.today()
year = today.year
#months = ['10', '11', '12', '01', '02', '03', '04']
months = ['11']
#days = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
days = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15']

schedule_list = []
for month in months:
    for day in days:
        time.sleep(1)
        schedule_response = requests.get(f"https://api.sportradar.us/ncaamb/trial/v8/en/games/{year}/{month}/{day}/schedule.json?api_key={api_key}")
        if schedule_response.status_code == 200:
            temp = []
            temp.append(schedule_response.json())
            schedule_list = schedule_list + temp

In [ ]:
schedule_dict = {'game_id': [], 'date': [], 'home_team': [], 'away_team': [], 'capacity': [], 'venue_id': [], 'home_id': [], 'away_id': []}

for date in schedule_list:
    if len(date['games']) > 0:
        for game in date['games']:
            
            schedule_dict['game_id'].append(game['id'])
            schedule_dict['date'].append(date['date'])
            schedule_dict['home_team'].append(game['home']['name'])
            schedule_dict['away_team'].append(game['away']['name'])

            try:
                schedule_dict['capacity'].append(game['venue']['capacity'])
            except:
                schedule_dict['capacity'].append(None)

            try:
                schedule_dict['venue_id'].append(game['venue']['id'])
            except:
                schedule_dict['venue_id'].append(None)

            schedule_dict['home_id'].append(game['home']['id'])
            schedule_dict['away_id'].append(game['away']['id'])
            
schedule_dict

In [ ]:
schedule_df = pd.DataFrame(schedule_dict)
schedule_df

In [ ]:
game_list = []
ix = 0
for game_id in schedule_df['game_id']:
    time.sleep(1)
    game_response = requests.get(f"https://api.sportradar.us/ncaamb/trial/v8/en/games/{game_id}/summary.json?api_key={api_key}")
    if game_response.status_code == 200:
        ix += 1
        temp = []
        temp.append(game_response.json())
        game_list = game_list + temp

In [ ]:
game_dict = {'game_id': [], 'attendance': [], 'venue_id': [], 'venue_time_zone': [], 'home_time_zone': [], 'away_time_zone': [], 'home_points': [], 'away_points': [], 'home_rank': [],
             'away_rank': [], 'home_field_goal_att': [], 'away_field_goal_att': [], 'home_field_goal_made': [], 'away_field_goal_made': [], 'home_field_goal_pct': [], 'away_field_goal_pct': [],
             'home_3pt_att': [], 'away_3pt_att': [], 'home_3pt_made': [], 'away_3pt_made': [], 'home_3pt_pct': [], 'away_3pt_pct': [], 'home_free_throw_att': [], 'away_free_throw_att': [], 'home_free_throw_made': [], 
             'away_free_throw_made': [], 'home_free_throw_pct': [], 'away_free_throw_pct': [], 'home_rebounds': [], 'away_rebounds': [], 'home_assists': [], 'away_assists': [], 'home_steals': [], 'away_steals': [], 
             'home_blocks': [], 'away_blocks': [], 'home_turnovers': [], 'away_turnovers': [], 'home_fouls': [], 'away_fouls': []}

for game in game_list:
    try:
       game_dict['game_id'].append(game['id'])
    except:
        game_dict['game_id'].append(None)

    try:
        game_dict['attendance'].append(game['attendance'])
    except:
        game_dict['attendance'].append(None)

    try:
        game_dict['venue_id'].append(game['venue']['id'])
    except:
        game_dict['venue_id'].append(None)

    try:
        game_dict['venue_time_zone'].append(game['time_zones']['venue'])
    except:
        game_dict['venue_time_zone'].append(None)

    try:
        game_dict['home_time_zone'].append(game['time_zones']['home'])
    except:
        game_dict['home_time_zone'].append(None)

    try:
        game_dict['away_time_zone'].append(game['time_zones']['away'])
    except:
        game_dict['away_time_zone'].append(None)

    try:
        game_dict['home_points'].append(game['home']['points'])
    except:
        game_dict['home_points'].append(None)

    try:
        game_dict['away_points'].append(game['away']['points'])
    except:
        game_dict['away_points'].append(None)

    try:
        game_dict['home_rank'].append(game['home']['rank'])
    except:
        game_dict['home_rank'].append(None)

    try:
        game_dict['away_rank'].append(game['away']['rank'])
    except:
        game_dict['away_rank'].append(None)

    try:
        game_dict['home_field_goal_att'].append(game['home']['statistics']['field_goals_att'])
    except:
        game_dict['home_field_goal_att'].append(None)

    try:
        game_dict['away_field_goal_att'].append(game['away']['statistics']['field_goals_att'])
    except:
        game_dict['away_field_goal_att'].append(None)

    try:
        game_dict['home_field_goal_made'].append(game['home']['statistics']['field_goals_made'])
    except:
        game_dict['home_field_goal_made'].append(None)

    try:
        game_dict['away_field_goal_made'].append(game['away']['statistics']['field_goals_made'])
    except:
        game_dict['away_field_goal_made'].append(None)

    try:
        game_dict['home_field_goal_pct'].append(game['home']['statistics']['field_goals_pct'])
    except:
        game_dict['home_field_goal_pct'].append(None)

    try:
        game_dict['away_field_goal_pct'].append(game['away']['statistics']['field_goals_pct'])
    except:
        game_dict['away_field_goal_pct'].append(None)

    try:
        game_dict['home_3pt_att'].append(game['home']['statistics']['three_points_att'])
    except:
        game_dict['home_3pt_att'].append(None)

    try:
        game_dict['away_3pt_att'].append(game['away']['statistics']['three_points_att'])
    except:
        game_dict['away_3pt_att'].append(None)

    try:
        game_dict['home_3pt_made'].append(game['home']['statistics']['three_points_made'])
    except:
        game_dict['home_3pt_made'].append(None)

    try:
        game_dict['away_3pt_made'].append(game['away']['statistics']['three_points_made'])
    except:
        game_dict['away_3pt_made'].append(None)

    try:
        game_dict['home_3pt_pct'].append(game['home']['statistics']['three_points_pct'])
    except:
        game_dict['home_3pt_pct'].append(None)

    try:
        game_dict['away_3pt_pct'].append(game['away']['statistics']['three_points_pct'])
    except:
        game_dict['away_3pt_pct'].append(None)

    try:
        game_dict['home_free_throw_att'].append(game['home']['statistics']['free_throws_att'])
    except:
        game_dict['home_free_throw_att'].append(None)

    try:
        game_dict['away_free_throw_att'].append(game['away']['statistics']['free_throws_att'])
    except:
        game_dict['away_free_throw_att'].append(None)

    try:
        game_dict['home_free_throw_made'].append(game['home']['statistics']['free_throws_made'])
    except:
        game_dict['home_free_throw_made'].append(None)

    try:
        game_dict['away_free_throw_made'].append(game['away']['statistics']['free_throws_made'])
    except:
        game_dict['away_free_throw_made'].append(None)

    try:
        game_dict['home_free_throw_pct'].append(game['home']['statistics']['free_throws_pct'])
    except:
        game_dict['home_free_throw_pct'].append(None)

    try:
        game_dict['away_free_throw_pct'].append(game['away']['statistics']['free_throws_pct'])
    except:
        game_dict['away_free_throw_pct'].append(None)

    try:
        game_dict['home_rebounds'].append(game['home']['statistics']['total_rebounds'])
    except:
        game_dict['home_rebounds'].append(None)

    try:
        game_dict['away_rebounds'].append(game['away']['statistics']['total_rebounds'])
    except:
        game_dict['away_rebounds'].append(None)

    try:
        game_dict['home_assists'].append(game['home']['statistics']['assists'])
    except:
        game_dict['home_assists'].append(None)

    try:
        game_dict['away_assists'].append(game['away']['statistics']['assists'])
    except:
        game_dict['away_assists'].append(None)

    try:
        game_dict['home_steals'].append(game['home']['statistics']['steals'])
    except:
        game_dict['home_steals'].append(None)

    try:
        game_dict['away_steals'].append(game['away']['statistics']['steals'])
    except:
        game_dict['away_steals'].append(None)

    try:
        game_dict['home_blocks'].append(game['home']['statistics']['blocks'])
    except:
        game_dict['home_blocks'].append(None)

    try:
        game_dict['away_blocks'].append(game['away']['statistics']['blocks'])
    except:
        game_dict['away_blocks'].append(None)

    try:
        game_dict['home_turnovers'].append(game['home']['statistics']['total_turnovers'])
    except:
        game_dict['home_turnovers'].append(None)

    try:
        game_dict['away_turnovers'].append(game['away']['statistics']['total_turnovers'])
    except:
        game_dict['away_turnovers'].append(None)

    try:
        game_dict['home_fouls'].append(game['home']['statistics']['personal_fouls'])
    except:
        game_dict['home_fouls'].append(None)

    try:
        game_dict['away_fouls'].append(game['away']['statistics']['personal_fouls'])
    except:
        game_dict['away_fouls'].append(None)

game_df = pd.DataFrame(game_dict)

In [ ]:
# Merge DataFrames
schedule_rank_df = pd.merge(schedule_df, game_df, how='inner', on=['game_id'])
schedule_rank_df

In [ ]:
# Drop Unnecessary Columns
schedule_rank_df.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y', 'venue_id_y'], inplace=True)
schedule_rank_df.rename(columns={'venue_id_x': 'venue_id'}, inplace=True)
schedule_rank_df

In [ ]:
schedule_rank_df.to_csv('schedule_rank_data_update.csv')

In [ ]:
# Generate Home and Away DataFrames
home_df = pd.read_csv('schedule_rank_data_update.csv')
home_df.drop(columns=['Unnamed: 0'], inplace=True)
home_df

In [ ]:
# Rename Columns
home_df.rename(columns={'home_team': 'team', 'away_team': 'opp', 'home_id': 'team_id', 'away_id': 'opp_id', 'home_time_zone': 'team_time_zone', 'away_time_zone': 'opp_time_zone', 
                        'home_points': 'team_points', 'away_points': 'opp_points', 'home_rank': 'team_rank', 'away_rank': 'opp_rank', 'home_field_goal_att': 'team_field_goal_att',
                        'away_field_goal_att': 'opp_field_goal_att', 'home_field_goal_made': 'team_field_goal_made','away_field_goal_made': 'opp_field_goal_made', 
                        'home_field_goal_pct': 'team_field_goal_pct','away_field_goal_pct': 'opp_field_goal_pct','home_3pt_att': 'team_3pt_att','away_3pt_att': 'opp_3pt_att', 
                        'home_3pt_made': 'team_3pt_made','away_3pt_made': 'opp_3pt_made','home_3pt_pct': 'team_3pt_pct','away_3pt_pct': 'opp_3pt_pct',
                        'home_free_throw_att': 'team_free_throw_att','away_free_throw_att': 'opp_free_throw_att', 'home_free_throw_made': 'team_free_throw_made',
                        'away_free_throw_made': 'opp_free_throw_made','home_free_throw_pct': 'team_free_throw_pct','away_free_throw_pct': 'opp_free_throw_pct', 'home_rebounds': 'team_rebounds',
                        'away_rebounds': 'opp_rebounds', 'home_assists':'team_assists', 'away_assists': 'opp_assists', 'home_steals': 'team_steals', 'away_steals': 'opp_steals',
                        'home_blocks': 'team_blocks', 'away_blocks': 'opp_blocks', 'home_turnovers': 'team_turnovers', 'away_turnovers': 'opp_turnovers', 'home_fouls': 'team_fouls',
                        'away_fouls': 'opp_fouls'}, inplace=True)


In [ ]:
away_df = pd.read_csv('schedule_rank_data_update.csv')
away_df.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
#Rename Columns
away_df.rename(columns={'home_team': 'opp', 'away_team': 'team', 'home_id': 'opp_id', 'away_id': 'team_id', 'home_time_zone': 'opp_time_zone', 'away_time_zone': 'team_time_zone', 
                        'home_points': 'opp_points', 'away_points': 'team_points', 'home_rank': 'opp_rank', 'away_rank': 'team_rank', 'home_field_goal_att': 'opp_field_goal_att',
                        'away_field_goal_att': 'team_field_goal_att', 'home_field_goal_made': 'opp_field_goal_made','away_field_goal_made': 'team_field_goal_made', 
                        'home_field_goal_pct': 'opp_field_goal_pct','away_field_goal_pct': 'team_field_goal_pct','home_3pt_att': 'opp_3pt_att','away_3pt_att': 'team_3pt_att', 
                        'home_3pt_made': 'opp_3pt_made','away_3pt_made': 'team_3pt_made','home_3pt_pct': 'opp_3pt_pct','away_3pt_pct': 'team_3pt_pct',
                        'home_free_throw_att': 'opp_free_throw_att','away_free_throw_att': 'team_free_throw_att', 'home_free_throw_made': 'opp_free_throw_made',
                        'away_free_throw_made': 'team_free_throw_made','home_free_throw_pct': 'opp_free_throw_pct','away_free_throw_pct': 'team_free_throw_pct', 'home_rebounds': 'opp_rebounds',
                        'away_rebounds': 'team_rebounds', 'home_assists':'opp_assists', 'away_assists': 'team_assists', 'home_steals': 'opp_steals', 'away_steals': 'team_steals',
                        'home_blocks': 'opp_blocks', 'away_blocks': 'team_blocks', 'home_turnovers': 'opp_turnovers', 'away_turnovers': 'team_turnovers', 'home_fouls': 'opp_fouls',
                        'away_fouls': 'team_fouls'}, inplace=True)


In [ ]:
# Combine Home and Away DataFrames
cbb_stats_df = pd.concat([home_df, away_df])
cbb_stats_df

In [ ]:
cbb_stats_df.sort_values(by='date', ascending=True, inplace=True)
cbb_stats_df

In [ ]:
cbb_stats_df['game_counter'] = 1
cbb_stats_df

In [ ]:
cbb_stats_df['total_team_points'] = cbb_stats_df.groupby('team_id')['team_points'].cumsum()
cbb_stats_df['total_opp_points'] = cbb_stats_df.groupby('opp_id')['opp_points'].cumsum()
cbb_stats_df['total_team_fg_att'] = cbb_stats_df.groupby('team_id')['team_field_goal_att'].cumsum()
cbb_stats_df['total_opp_fg_att'] = cbb_stats_df.groupby('opp_id')['opp_field_goal_att'].cumsum()
cbb_stats_df['total_team_fg_made'] = cbb_stats_df.groupby('team_id')['team_field_goal_made'].cumsum()
cbb_stats_df['total_opp_fg_made'] = cbb_stats_df.groupby('opp_id')['opp_field_goal_made'].cumsum()
cbb_stats_df['total_team_fg_pct'] = cbb_stats_df.groupby('team_id')['team_field_goal_pct'].cumsum()/cbb_stats_df.groupby('team_id')['game_counter'].cumsum() * 100
cbb_stats_df['total_opp_fg_pct'] = cbb_stats_df.groupby('opp_id')['opp_field_goal_pct'].cumsum()/cbb_stats_df.groupby('team_id')['game_counter'].cumsum() * 100
cbb_stats_df['total_team_3pt_att'] = cbb_stats_df.groupby('team_id')['team_3pt_att'].cumsum()
cbb_stats_df['total_opp_3pt_att'] = cbb_stats_df.groupby('opp_id')['opp_3pt_att'].cumsum()
cbb_stats_df['total_team_3pt_made'] = cbb_stats_df.groupby('team_id')['team_3pt_made'].cumsum()
cbb_stats_df['total_opp_3pt_made'] = cbb_stats_df.groupby('opp_id')['opp_3pt_made'].cumsum()
cbb_stats_df['total_team_3pt_pct'] = cbb_stats_df.groupby('team_id')['team_3pt_pct'].cumsum()/cbb_stats_df.groupby('team_id')['game_counter'].cumsum() * 100
cbb_stats_df['total_opp_3pt_pct'] = cbb_stats_df.groupby('opp_id')['opp_3pt_pct'].cumsum()/cbb_stats_df.groupby('team_id')['game_counter'].cumsum() * 100
cbb_stats_df['total_team_ft_att'] = cbb_stats_df.groupby('team_id')['team_free_throw_att'].cumsum()
cbb_stats_df['total_opp_ft_att'] = cbb_stats_df.groupby('opp_id')['opp_free_throw_att'].cumsum()
cbb_stats_df['total_team_ft_made'] = cbb_stats_df.groupby('team_id')['team_free_throw_made'].cumsum()
cbb_stats_df['total_opp_ft_made'] = cbb_stats_df.groupby('opp_id')['opp_free_throw_made'].cumsum()
cbb_stats_df['total_team_ft_pct'] = cbb_stats_df.groupby('team_id')['team_free_throw_pct'].cumsum()/cbb_stats_df.groupby('team_id')['game_counter'].cumsum() * 100
cbb_stats_df['total_opp_ft_pct'] = cbb_stats_df.groupby('opp_id')['opp_free_throw_pct'].cumsum()/cbb_stats_df.groupby('team_id')['game_counter'].cumsum() * 100
cbb_stats_df['total_team_rebounds'] = cbb_stats_df.groupby('team_id')['team_rebounds'].cumsum()
cbb_stats_df['total_opp_rebounds'] = cbb_stats_df.groupby('opp_id')['opp_rebounds'].cumsum()
cbb_stats_df['total_team_assists'] = cbb_stats_df.groupby('team_id')['team_assists'].cumsum()
cbb_stats_df['total_opp_assists'] = cbb_stats_df.groupby('opp_id')['opp_assists'].cumsum()
cbb_stats_df['total_team_steals'] = cbb_stats_df.groupby('team_id')['team_steals'].cumsum()
cbb_stats_df['total_opp_steals'] = cbb_stats_df.groupby('opp_id')['opp_steals'].cumsum()
cbb_stats_df['total_team_blocks'] = cbb_stats_df.groupby('team_id')['team_blocks'].cumsum()
cbb_stats_df['total_opp_blocks'] = cbb_stats_df.groupby('opp_id')['opp_blocks'].cumsum()
cbb_stats_df['total_team_turnovers'] = cbb_stats_df.groupby('team_id')['team_turnovers'].cumsum()
cbb_stats_df['total_opp_turnovers'] = cbb_stats_df.groupby('opp_id')['opp_turnovers'].cumsum()
cbb_stats_df['total_team_fouls'] = cbb_stats_df.groupby('team_id')['team_fouls'].cumsum()
cbb_stats_df['total_opp_fouls'] = cbb_stats_df.groupby('opp_id')['opp_fouls'].cumsum()

cbb_stats_df

In [ ]:
cbb_stats_df.to_csv('cbb_raw_data_update.csv')

In [ ]:
cbb_norm_df = pd.read_csv('cbb_raw_data_update.csv')
cbb_norm_df.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
cbb_norm_df['venue_code'] = cbb_norm_df['venue_id'].astype("category").cat.codes
cbb_norm_df['venue_time_zone_code'] = cbb_norm_df['venue_time_zone'].astype("category").cat.codes
cbb_norm_df['team_time_zone_code'] = cbb_norm_df['team_time_zone'].astype("category").cat.codes
cbb_norm_df['opp_time_zone_code'] = cbb_norm_df['opp_time_zone'].astype("category").cat.codes
cbb_norm_df['team_code'] = cbb_norm_df['team_id'].astype("category").cat.codes
cbb_norm_df['opp_code'] = cbb_norm_df['opp_id'].astype("category").cat.codes
cbb_norm_df['team_rank'] = np.where(cbb_norm_df['team_rank'] == 0, 50, cbb_norm_df['team_rank'])
cbb_norm_df['opp_rank'] = np.where(cbb_norm_df['opp_rank'] == 0, 50, cbb_norm_df['opp_rank'])
cbb_norm_df

In [ ]:
cbb_norm_df['capacity'] = cbb_norm_df['capacity']/cbb_norm_df['capacity'].max()
cbb_norm_df['attendance'] = cbb_norm_df['attendance']/cbb_norm_df['attendance'].max()
cbb_norm_df['total_team_points'] = cbb_norm_df['total_team_points']/cbb_norm_df['total_team_points'].max()
cbb_norm_df['total_opp_points'] = cbb_norm_df['total_opp_points']/cbb_norm_df['total_opp_points'].max()
cbb_norm_df['total_team_fg_att'] = cbb_norm_df['total_team_fg_att']/cbb_norm_df['total_team_fg_att'].max()
cbb_norm_df['total_opp_fg_att'] = cbb_norm_df['total_opp_fg_att']/cbb_norm_df['total_opp_fg_att'].max()
cbb_norm_df['total_team_fg_made'] = cbb_norm_df['total_team_fg_made']/cbb_norm_df['total_team_fg_made'].max() 
cbb_norm_df['total_opp_fg_made'] = cbb_norm_df['total_opp_fg_made']/cbb_norm_df['total_opp_fg_made'].max()
cbb_norm_df['total_team_fg_pct'] = cbb_norm_df['total_team_fg_pct']/cbb_norm_df['total_team_fg_pct'].max()
cbb_norm_df['total_opp_fg_pct'] = cbb_norm_df['total_opp_fg_pct']/cbb_norm_df['total_opp_fg_pct'].max()
cbb_norm_df['total_team_3pt_att'] = cbb_norm_df['total_team_3pt_att']/cbb_norm_df['total_team_3pt_att'].max()
cbb_norm_df['total_opp_3pt_att'] = cbb_norm_df['total_opp_3pt_att']/cbb_norm_df['total_opp_3pt_att'].max()
cbb_norm_df['total_team_3pt_made'] = cbb_norm_df['total_team_3pt_made']/cbb_norm_df['total_team_3pt_made'].max()
cbb_norm_df['total_opp_3pt_made'] = cbb_norm_df['total_opp_3pt_made']/cbb_norm_df['total_opp_3pt_made'].max()
cbb_norm_df['total_team_3pt_pct'] = cbb_norm_df['total_team_3pt_pct']/cbb_norm_df['total_team_3pt_pct'].max()
cbb_norm_df['total_opp_3pt_pct'] = cbb_norm_df['total_opp_3pt_pct']/cbb_norm_df['total_opp_3pt_pct'].max()
cbb_norm_df['total_team_ft_att'] = cbb_norm_df['total_team_ft_att']/cbb_norm_df['total_team_ft_att'].max()
cbb_norm_df['total_opp_ft_att'] = cbb_norm_df['total_opp_ft_att']/cbb_norm_df['total_opp_ft_att'].max()
cbb_norm_df['total_team_ft_made'] = cbb_norm_df['total_team_ft_made']/cbb_norm_df['total_team_ft_made'].max()
cbb_norm_df['total_opp_ft_made'] = cbb_norm_df['total_opp_ft_made']/cbb_norm_df['total_opp_ft_made'].max()
cbb_norm_df['total_team_ft_pct'] = cbb_norm_df['total_team_ft_pct']/cbb_norm_df['total_team_ft_pct'].max()
cbb_norm_df['total_opp_ft_pct'] = cbb_norm_df['total_opp_ft_pct']/cbb_norm_df['total_opp_ft_pct'].max()
cbb_norm_df['total_team_rebounds'] = cbb_norm_df['total_team_rebounds']/cbb_norm_df['total_team_rebounds'].max()
cbb_norm_df['total_opp_rebounds'] = cbb_norm_df['total_opp_rebounds']/cbb_norm_df['total_opp_rebounds'].max()
cbb_norm_df['total_team_assists'] = cbb_norm_df['total_team_assists']/cbb_norm_df['total_team_assists'].max()
cbb_norm_df['total_opp_assists'] = cbb_norm_df['total_opp_assists']/cbb_norm_df['total_opp_assists'].max()
cbb_norm_df['total_team_steals'] = cbb_norm_df['total_team_steals']/cbb_norm_df['total_team_steals'].max()
cbb_norm_df['total_opp_steals'] = cbb_norm_df['total_opp_steals']/cbb_norm_df['total_opp_steals'].max()
cbb_norm_df['total_team_blocks'] = cbb_norm_df['total_team_blocks']/cbb_norm_df['total_team_blocks'].max()
cbb_norm_df['total_opp_blocks'] = cbb_norm_df['total_opp_blocks']/cbb_norm_df['total_opp_blocks'].max()
cbb_norm_df['total_team_turnovers'] = cbb_norm_df['total_team_turnovers'] /cbb_norm_df['total_team_turnovers'] .max()
cbb_norm_df['total_opp_turnovers'] = cbb_norm_df['total_opp_turnovers']/cbb_norm_df['total_opp_turnovers'].max()
cbb_norm_df['total_team_fouls'] = cbb_norm_df['total_team_fouls']/cbb_norm_df['total_team_fouls'].max()
cbb_norm_df['total_opp_fouls'] = cbb_norm_df['total_opp_fouls']/cbb_norm_df['total_opp_fouls'].max()
cbb_norm_df['venue_code'] = cbb_norm_df['venue_code']/cbb_norm_df['venue_code'].max()
cbb_norm_df['venue_time_zone_code'] = cbb_norm_df['venue_time_zone_code']/cbb_norm_df['venue_time_zone_code'].max()
cbb_norm_df['team_time_zone_code'] = cbb_norm_df['team_time_zone_code']/cbb_norm_df['team_time_zone_code'].max()
cbb_norm_df['opp_time_zone_code'] = cbb_norm_df['opp_time_zone_code']/cbb_norm_df['opp_time_zone_code'].max()
cbb_norm_df['team_code'] = cbb_norm_df['team_code']/cbb_norm_df['team_code'].max()
cbb_norm_df['opp_code'] = cbb_norm_df['opp_code']/cbb_norm_df['opp_code'].max()
cbb_norm_df['team_rank'] = cbb_norm_df['team_rank']/cbb_norm_df['team_rank'].max()
cbb_norm_df['opp_rank'] = cbb_norm_df['opp_rank']/cbb_norm_df['opp_rank'].max()
cbb_norm_df

In [ ]:
# Concatenate New Data onto Old Dataset
updated_raw_df = pd.concat([cbb_stats_df, old_raw_df])
updated_training_df = pd.concat([cbb_norm_df, old_training_df])

# Save DFs to Excel
updated_raw_df.to_excel(f"{cbb_raw_path}/{cbb_raw_name}")
updated_training_df.to_excel(f"{cbb_training_path}/{cbb_training_name}")

In [ ]:
data_df = pd.read_excel(f"{cbb_training_path}/{cbb_training_name}", index_col=0)
data_df = data_df.dropna(axis = 0).reset_index()

# Define Metrics for Input Variables
X = data_df[['capacity', 'team_rank', 'opp_rank', 'total_team_points','total_opp_points', 'total_team_fg_att',
                   'total_opp_fg_att', 'total_team_fg_made', 'total_opp_fg_made', 'total_team_fg_pct', 'total_opp_fg_pct',
                   'total_team_3pt_att','total_opp_3pt_att', 'total_team_3pt_made', 'total_opp_3pt_made', 'total_team_3pt_pct', 'total_opp_3pt_pct',
                    'total_team_ft_att','total_opp_ft_att', 'total_team_ft_made', 'total_opp_ft_made', 'total_team_ft_pct', 'total_opp_ft_pct', 
                    'total_team_rebounds', 'total_opp_rebounds', 'total_team_assists', 'total_opp_assists', 'total_team_steals', 'total_opp_steals',
                    'total_team_blocks', 'total_opp_blocks', 'total_team_turnovers', 'total_opp_turnovers', 'total_team_fouls', 'total_opp_fouls',
                    'venue_code', 'team_code', 'opp_code']]
X = X.reset_index(drop=True)

In [ ]:
# Model Output Path and File Names
model_path = "/Users/matthew.yough/Documents/GitHub/ncaab_game_predictor"
model_file = "CBB_Score_Model.pkl"

y_team = data_df['team_points']
#y_opp = data_df['opp_points']

# Home Linear Regression Model
X_train, X_test, y_train, y_test = train_test_split(X, y_team, test_size = 0.20)

regr = LinearRegression()
regr.fit(X_train, y_train.values.ravel())
print(regr.score(X_test, y_test))

# Away Linear Regression model
'''
X_train, X_test, y_train, y_test = train_test_split(X_opp, y_opp, test_size = 0.20)

regr_away = LinearRegression()
regr_away.fit(X_train, y_train.values.ravel())
print(regr_away.score(X_test, y_test))
'''

# Save Models
joblib.dump(regr, f"{model_path}/{model_file}")